# Sensitivity Analysis with the OpenCL RAMP model

### Import opencl modules

In [1]:
import multiprocessing as mp
import numpy as np
import yaml # pyyaml library for reading the parameters.yml file
import os
import pandas as pd
import unittest
import pickle
import copy
import matplotlib.pyplot as plt

from microsim.opencl.ramp.run import run_headless
from microsim.opencl.ramp.snapshot_convertor import SnapshotConvertor
from microsim.opencl.ramp.snapshot import Snapshot
from microsim.opencl.ramp.params import Params, IndividualHazardMultipliers, LocationHazardMultipliers
from microsim.opencl.ramp.simulator import Simulator
from microsim.opencl.ramp.disease_statuses import DiseaseStatus

import sys
sys.path.append('..')
#import experiments_functions  # For the ones outside the class
from opencl_runner import OpenCLRunner # Some additional notebook-specific functions required (functions.py)

# Useful for connecting to this kernel
#%connect_info

### Setup params for all runs

Read the parameters file

Prepare the parameters for the OpenCL model. (See [main.py](https://github.com/Urban-Analytics/RAMP-UA/blob/052861cc51be5bc1827c85bb827209f0df73c685/microsim/main.py#L262) for an example of how this is done in the code). 

In [2]:
PARAMETERS_FILENAME = "default.yml"  # Creates default parameters using the default.yml file
PARAMS = Functions.create_parameters(
    parameters_file=os.path.join("../../","model_parameters", PARAMETERS_FILENAME))

### Get snapshot path
**NB** this is the path to the OpenCL snapshot file generated by running `microsim/main.py`. To run with new population data just re-run `main.py --opencl` without the `--use-cache` option, so that it regenerates a new snapshot file and writes it to this location.

In [3]:
OPENCL_DIR = "../../microsim/opencl"
SNAPSHOT_FILEPATH = os.path.join(OPENCL_DIR, "snapshots", "cache.npz")
assert os.path.isfile(SNAPSHOT_FILEPATH), f"Snapshot doesn't exist: {SNAPSHOT_FILEPATH}"

## Run Default OpenCL simulation for multiple repetitions

This shows what happens with the 'default' (manually calibrated) model

In [ ]:
iterations = 100
repetitions = 10
num_seed_days = 10
use_gpu=False

results = Functions.run_opencl_model_multi(
    repetitions=10, 
    iterations=100, 
    params=Functions.create_parameters(parameters_file=os.path.join("../..", "model_parameters", "default.yml")),
    num_seed_days=10,
    store_detailed_counts=True,
    opencl_dir=os.path.join("../..", "microsim", "opencl"),
    snapshot_filepath=os.path.join("../..", "microsim", "opencl", "snapshots", "cache.npz"),
    multiprocess=False
)

summaries = [x[0] for x in results]
final_results = [x[1] for x in results]
print("Finished")

# To make it convenient to reload later:
pickle.dump( summaries, open( "./summaries.pkl", "wb" ) )

## Plot output summary data

### Total counts of disease status

In [ ]:
def plot_summaries(summaries, plot_type="error_bars"):

    #fig, ax = plt.subplots(1, len(DiseaseStatus), sharey=True)
    fig, ax = plt.subplots(1, 1, figsize=(10,7))
    
    # Work out the number of repetitions and iterations
    iters, reps = _get_iters_and_reps(summaries)
    x = range(iters)

    for d, disease_status in enumerate(DiseaseStatus):
        if disease_status==DiseaseStatus.Susceptible or disease_status==DiseaseStatus.Recovered:
            continue
        # Calculate the mean and standard deviation
        matrix = np.zeros(shape=(reps,iters))
        for rep in range(reps):
            matrix[rep] = summaries[rep].total_counts[d]
        mean = np.mean(matrix, axis=0)
        sd = np.std(matrix, axis=0)
        if plot_type == "error_bars":
            ax.errorbar(x, mean, sd, label=f"{disease_status}" )
        elif plot_type == "lines":
            for rep in range(reps):
                ax.plot(x, matrix[rep], label=f"{disease_status} {rep}", 
                        color=plt.cm.get_cmap("hsv", len(DiseaseStatus))(d) )
                
    ax.legend() 
    ax.set_title("Disease Status")
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Number of cases")

def _get_iters_and_reps(summaries):
    reps = len(summaries)
    iters = len(summaries[0].total_counts[0])
    return (iters, reps)

In [ ]:
plot_summaries(summaries=summaries, plot_type="error_bars")

In [ ]:
#plot_summaries(summaries=summaries, plot_type="lines")

### Disease statuses by age

In [ ]:
   
def plot_disease_status_by_age(summaries):

    #fig, ax = plt.subplots(1, len(DiseaseStatus), sharey=True)
    fig, ax = plt.subplots(int(len(DiseaseStatus)/2), int(len(DiseaseStatus)/2), 
                           figsize=(15,11), tight_layout=True)
    iters, reps = _get_iters_and_reps(summaries)
    x = range(iters)
    age_thresholds = summaries[0].age_thresholds

    for d, disease_status in enumerate(DiseaseStatus):
        lower_age_bound = 0
        for age_idx in range(len(age_thresholds)):
            matrix = np.zeros(shape=(reps, iters))
            for rep in range(reps):
                #matrix[age_idx][rep][it] = summaries[rep].age_counts[str(disease_status)][age_idx][it]
                matrix[rep] = summaries[rep].age_counts[str(disease_status)][age_idx]
            mean = np.mean(matrix, axis=0)
            sd = np.std(matrix, axis=0)
            ax.flat[d].errorbar(x, mean, sd, label=f"{lower_age_bound} - {age_thresholds[age_idx]}" )
            lower_age_bound = age_thresholds[age_idx]
                
            ax.flat[d].legend() 
            ax.flat[d].set_title(f"{str(disease_status)}")
            ax.flat[d].set_xlabel("Iteration")
            ax.flat[d].set_ylabel("Number of cases")
    #fig.set_title(f"Num {disease_status} people by age group")

In [ ]:
plot_disease_status_by_age(summaries)

### Plot MSOA geodata

#### Load MSOA shapes

In [ ]:
from microsim.load_msoa_locations import load_osm_shapefile, load_msoa_shapes
import pandas as pd

data_dir = ("../devon_data")

osm_buildings = load_osm_shapefile(data_dir)

devon_msoa_shapes = load_msoa_shapes(data_dir, visualize=False)

devon_msoa_shapes.plot()
plt.show()

In [ ]:
import pandas as pd

def plot_msoa_choropleth(msoa_shapes, summary, disease_status, timestep):
    # get dataframes for all statuses
    msoa_data = summary.get_area_dataframes()
    
    msoa_data_for_status = msoa_data[disease_status]

    # add "Code" column so dataframes can be merged
    msoa_data_for_status["Code"] = msoa_data_for_status.index
    msoa_shapes = pd.merge(msoa_shapes, msoa_data_for_status, on="Code")

    msoa_shapes.plot(column=f"Day{timestep}", legend=True)
    plt.show()

### Plot disease status by MSOA for a given timestep and status

In [ ]:
disease_status = "exposed"

plot_msoa_choropleth(devon_msoa_shapes, summaries[0], disease_status, 99)

## Observation Data

Read the real observations (number of hospital admissions in Devon) that will be used to calibrate the model. See the [README](./observation_data/README.md) for information about how these observations were obtained. They aren't the raw cases, it's actually a model that was fitted to the lagged cases.

In [4]:
observations = pd.read_csv("observation_data/gam_cases.csv", header=0, names=["Day", "Cases"], )
observations

,Day,Cases
0,1,10
1,2,11
2,3,13
3,4,16
4,5,18
...,...,...
98,99,10
99,100,9
100,101,9
101,102,8


## Calculating Error: Fitness Function

To calibrate the model we need a fitness function that tells us, for a given result, how similar it is to the observations. There are lots of ways to do this. For now, just take the **Euclidean distance (L2 norm)** between the observed number of cases and the simulated number of cases.

Note that the model is seeded using the first few days of cases, so at the beginning of a run the simulated data will be identical to the observations. This doesn't matter though because the relative difference between different parameter combinations will be the same regardless.

In [5]:
# The fitness function is defined in experiments_functions.py (so that it can be tested)
#fit = Functions.fit_l2

## Calibration Parameters

Which parameters will we try to calibrate on?

To begin with lets just try the `current_risk_beta` (a general multiplier for risk at locations).

Create a function that takes a value of that parameter as an argument, runs a model, and returns the result (the number of cases per day).

In [6]:
ITERATIONS = 100  # Number of iterations to run for
NUM_SEED_DAYS = 10  # Number of days to seed the population
USE_GPU = False
STORE_DETAILED_COUNTS = False
REPETITIONS = 5 

snapshot_filepath =             os.path.join("../..", "microsim", "opencl", "snapshots", "cache.npz"),


assert ITERATIONS < len(observations), \
    f"Have more iterations ({ITERATIONS}) than observations ({len(observations)})."


In [20]:
# DELETE
def __run_model_with_params(input_params: List, return_full_details=False):
    """Run a model REPETITIONS times using the provided parameter values.
    
    :param input_params: The parameter values to pass, as a list. These need to correspond to specific parameters. Currently they are:
       input_params[0] -> current_risk_beta
    :param return_details: If True then rather than just returning the fitness, 
        return a tuple of (fitness, summaries_list, final_results_list).
    :return: The mean fitness across all model runs
    
    """
    
    current_risk_beta = input_params[0]
    proportion_asymptomatic = input_params[1]

    params = Functions.create_parameters(parameters_file=os.path.join("../..", "model_parameters", "default.yml"), 
                    current_risk_beta=current_risk_beta,
                    proportion_asymptomatic=proportion_asymptomatic)
    
    results = Functions.run_opencl_model_multi(
        repetitions=REPETITIONS, iterations=ITERATIONS, params=params,
        opencl_dir=os.path.join("../..", "microsim", "opencl"),
        snapshot_filepath=os.path.join("../..", "microsim", "opencl", "snapshots", "cache.npz"),
        multiprocess=False
    )
    
    summaries = [x[0] for x in results]
    final_results = [x[1] for x in results]

    # Get mean cases per day from the summary object
    sim = Functions.get_mean_total_counts(summaries, DiseaseStatus.Exposed.value)
    # Compare these to the observations
    obs = observations.loc[:ITERATIONS-1,"Cases"].values
    assert len(sim) == len(obs)
    fitness = fit(sim,obs)
    if return_full_details:
        return ( fitness, sim, obs, params)
    else:
        return fitness


Check that function works by calling it once with a couple of new parameters

In [21]:
(fitness, sim, obs, out_params) = run_model_with_params(np.array([
    0.005,  # current_risk_beta
    0.13  # proportion_asymptomatic
]) , return_full_details=True)

# Make sure that the parameter value used in the model was set correctly
assert out_params.proportion_asymptomatic == 0.13, f"Proportion_asymptomatic should be 0.13, not {out_params.proportion_asymptomatic}"

print(f"fitness: {fitness}")
#list(zip(obs,sim))

Running models: 100%|██████████| 5/5 [00:15<00:00,  3.03s/it]

.. finished, took 15.14s)
fitness: 43847.86587645971


### Calibration Method: XXXX

Lots of different methods are avialbale. Simulated annealing? Latin-Hypercube sampling? GA? ABC?



Start with a simple minimisation algorithm [Nelder-Mead Simplex algorithm](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#nelder-mead-simplex-algorithm-method-nelder-mead)

In [22]:
from scipy.optimize import minimize

# x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

x0 = np.array([  # initial guesses for each variable:
    0.005,  # current_risk_beta
    0.4  # proportion_asymptomatic
])  

optimisation_result = minimize(run_model_with_params, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})

pickle.dump( optimisation_result, open( "./optimisation_result.pkl", "wb" ) )

Running models: 100%|██████████| 5/5 [00:14<00:00,  2.88s/it]

.. finished, took 14.42s)



Running models: 100%|██████████| 5/5 [00:14<00:00,  3.00s/it]

.. finished, took 14.98s)



Running models: 100%|██████████| 5/5 [00:15<00:00,  3.13s/it]

...

.. finished, took 14.58s)



Running models: 100%|██████████| 5/5 [00:13<00:00,  2.71s/it]

.. finished, took 13.57s)



Running models: 100%|██████████| 5/5 [00:14<00:00,  2.83s/it]

.. finished, took 14.17s)
Optimization terminated successfully.
         Current function value: 973.483498
         Iterations: 56
         Function evaluations: 142


In [23]:
optimisation_result.x

array([0.00096495, 0.51683857])

# Unit Tests

I've not put tests into a notebook before. Not sure how well this will work. Uses a small ipython extension: https://github.com/akaihola/ipython_pytest/